### The Requirements:
Step 1: Choose an ETF with a minimum of 100 assets, identify those assets

Step 2: Retrieve historical data for your chosen ETF

Step 3: Calculate the price momentum factors for each asset in your ETF

Step 4: Using the price momentum factors, calculate the monthly z-factor score for each asset

Step 5: Identify long and short baskets (10 to 15 assets in each) using calculated z-factors

Step 6: Create a backtest to validate performance of your algorithm based on monthly restructuring over the previous 5 years.

Step 7: Chart:

1. Monthly portfolio return bar chart (pos/neg coloring) vs ETF

2. Monthly return for/ long picks vs short picks vs ETF

3. Cumulative portfolio return vs ETF

In [3]:
# Import Libraries
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt


In [4]:
# Step 1: Choose an ETF with a minimum of 100 assets, identify those assets
# etf = ["SPY"]

# Get the list of S&P 500 constituents
# SPY_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
tickers_list = pd.read_excel("https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx", header=4).Ticker.dropna().to_list()

print(f'{len(tickers_list)} tickers')


504 tickers


In [5]:
# Step 2: Retrieve historical data for your chosen ETF

# GET ADJ CLOSE DATA FOR PREVIOUS 5 YEARS
data = yf.download(tickers_list, period = '5y')['Adj Close']
# DROP NA
sp500 = data.dropna(how= 'all', axis= 1)
# PRINT OUT TAIL
#sp500.tail()

[*********************100%%**********************]  504 of 504 completed


3 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (period=5y)')
['BRK.B', '-']: Exception('%ticker%: No data found, symbol may be delisted')


In [6]:
sp500[-252:]

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-05,151.628784,14.11,145.814972,157.574310,98.510002,103.342644,59.169998,288.161743,334.089996,164.703476,...,85.101929,66.510796,103.322365,30.151529,110.601547,128.492340,121.350327,261.799988,46.111526,153.925903
2022-12-06,150.288254,14.33,142.115646,157.362854,93.120003,101.823029,60.540001,280.946381,331.149994,163.398331,...,84.744736,67.130592,100.450417,29.649332,109.387329,128.207687,120.467094,254.210007,44.892551,151.666519
2022-12-07,152.651535,13.55,140.156601,158.977631,91.500000,102.754410,60.150002,282.206329,326.679993,162.907654,...,84.298241,66.694794,100.228012,29.304688,109.555153,125.901085,123.245804,252.539993,44.806843,148.891830
2022-12-08,155.183640,13.60,141.857086,159.544724,94.830002,104.823021,60.560001,287.580963,332.579987,166.479645,...,85.756798,67.605110,100.972580,29.816732,109.910522,127.451912,124.883263,252.820007,45.121109,152.290817
2022-12-09,151.877014,13.53,141.369812,156.728470,94.699997,105.401451,60.040001,283.899475,330.640015,165.527756,...,84.328011,67.634155,100.121643,29.797037,109.801941,125.351418,123.454216,251.449997,45.225864,152.003433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-29,127.589996,12.23,189.369995,138.500000,126.480003,103.629997,82.379997,333.339996,617.390015,181.853149,...,83.669998,60.000000,102.339996,31.330000,103.500000,125.519997,113.919998,236.619995,35.439999,175.789993
2023-11-30,127.800003,12.43,189.949997,142.389999,126.339996,104.290001,83.690002,333.140015,611.010010,182.520004,...,84.419998,60.840000,102.739998,31.750000,105.129997,125.550003,116.309998,236.979996,35.630001,176.669998
2023-12-01,128.789993,13.02,191.240005,143.410004,135.020004,104.879997,82.629997,338.059998,612.469971,183.070007,...,85.150002,61.430000,102.989998,32.549999,106.199997,127.330002,117.970001,241.220001,38.320000,179.130005


In [7]:
# window = 20 days
lag = 20

# 
returns = sp500.pct_change()
returns.tail()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-29 00:00:00,0.023094,0.004930,-0.005410,0.003042,-0.008467,0.015383,-0.024512,0.002345,-0.009514,-0.002675,...,-0.014952,-0.015909,-0.015014,-0.004765,0.008379,-0.006883,0.007339,0.011672,0.025760,-0.006668
2023-11-30 00:00:00,0.001646,0.016353,0.003063,0.028087,-0.001107,0.006369,0.015902,-0.000600,-0.010334,0.003667,...,0.008964,0.014000,0.003909,0.013406,0.015749,0.000239,0.020980,0.001521,0.005361,0.005006
2023-12-01 00:00:00,0.007746,0.047466,0.006791,0.007163,0.068704,0.005657,-0.012666,0.014769,0.002389,0.003013,...,0.008647,0.009698,0.002433,0.025197,0.010178,0.014178,0.014272,0.017892,0.075498,0.013924
2023-12-04 00:00:00,0.000699,0.025346,-0.009465,0.005160,-0.009776,0.002956,-0.007745,-0.004822,-0.012915,0.001038,...,-0.014797,0.000651,-0.005437,0.005530,0.002542,-0.013194,-0.009155,-0.007669,0.009395,0.016692
2023-12-05 00:00:00,-0.017613,-0.020225,0.018899,0.000971,0.001421,-0.007986,0.001220,-0.001308,-0.009587,-0.011786,...,-0.019311,-0.005043,-0.014400,-0.017873,-0.006856,-0.009312,-0.008641,-0.022831,-0.027146,-0.013837


lag - last 20 entry

- its inplace to stop from ovefit 

In [8]:



lagged_returns.tail()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-29 00:00:00,0.021746,-0.002683,0.002819,-0.005004,0.011458,0.016667,0.045849,0.014998,0.009716,0.009302,...,-0.003406,0.009023,-0.000283,0.014005,0.047832,0.008259,0.009670,-0.001621,0.029019,0.002394
2023-11-30 00:00:00,-0.004934,-0.000897,0.018739,0.009137,0.009975,0.004865,0.038302,0.011949,0.023381,0.012140,...,-0.009569,0.021427,-0.001984,0.014140,0.010049,0.003641,0.006321,-0.054624,0.005511,-0.035414
2023-12-01 00:00:00,0.015652,0.023339,0.020693,0.005545,-0.033230,0.009999,-0.045111,0.024880,0.026097,0.032027,...,0.032321,0.008755,0.032847,-0.074578,-0.001799,0.024485,0.048825,0.045507,0.071244,0.062533
2023-12-04 00:00:00,0.043553,0.050877,-0.005181,-0.012844,0.061818,-0.001251,-0.004538,0.017428,0.008860,0.021784,...,0.033538,-0.000655,-0.012190,0.023476,0.027781,0.016657,-0.013158,0.035266,0.046043,0.008203
2023-12-05 00:00:00,-0.013667,-0.026711,0.014605,-0.001556,-0.036448,-0.008765,-0.004559,-0.001276,0.003176,-0.002084,...,0.025658,-0.007701,-0.017721,-0.022253,-0.007944,0.000396,0.005057,-0.038637,-0.016686,0.005794


In [9]:
# PAST 52 WEEK
#last_52_week = sp500[-252:]
last_52_week = lagged_returns[-252:]

len(last_52_week)


252

In [10]:
returns / len(last_52_week) * 100

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-07,-0.009161,-0.036202,-0.014150,-0.014805,NaN,-0.008820,-0.000143,-0.004485,-0.019997,-0.013784,...,-0.026194,-0.000149,-0.003797,-0.001741,-0.006769,-0.004640,-0.010508,-0.016647,-0.006745,-0.010183
2018-12-10,0.001921,0.003192,0.002614,0.002966,NaN,0.005503,-0.002432,0.000680,0.010154,0.009459,...,0.004725,0.002014,-0.005622,0.001639,0.001269,-0.000746,0.004260,0.006072,-0.007741,0.003739
2018-12-11,0.002586,0.003049,-0.002269,0.002537,NaN,0.005148,-0.004750,0.001459,0.002032,0.000718,...,-0.002615,0.000445,0.000726,0.005877,-0.002890,-0.001979,0.003250,0.004964,-0.006998,0.003927
2018-12-12,0.003909,0.002909,0.001106,0.001935,NaN,0.003811,-0.000874,0.000802,0.000340,0.001970,...,0.014513,-0.004301,-0.003416,0.012012,0.001274,0.002872,0.003330,0.008265,0.002466,0.004110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-29,0.009164,0.001956,-0.002147,0.001207,-0.003360,0.006104,-0.009727,0.000931,-0.003775,-0.001061,...,-0.005933,-0.006313,-0.005958,-0.001891,0.003325,-0.002732,0.002912,0.004632,0.010222,-0.002646
2023-11-30,0.000653,0.006489,0.001215,0.011145,-0.000439,0.002527,0.006310,-0.000238,-0.004101,0.001455,...,0.003557,0.005556,0.001551,0.005320,0.006250,0.000095,0.008325,0.000604,0.002127,0.001987
2023-12-01,0.003074,0.018836,0.002695,0.002843,0.027263,0.002245,-0.005026,0.005861,0.000948,0.001196,...,0.003431,0.003848,0.000966,0.009999,0.004039,0.005626,0.005664,0.007100,0.029960,0.005526


In [11]:
from scipy.stats import linregress

def slopes_of_linear_regression(data):
    
    slopes_dict = {}

    # Create an array of numbers from 1 to trading_days for the x-axis
    x_values = np.arange(1, len(data) + 1)


    # Calculate the linear regression for each column
    for column in data.columns:
        y_values = data[column]
        slope, _, _, _, _ = linregress(x_values, y_values)
        # slopes_dict[column] = slope
        # Multiply the slope by 100 to express it as a percentage change
        slopes_dict[column] = slope * 100

    return slopes_dict

slopes = slopes_of_linear_regression(last_52_week)
print("Slopes for each column:", slopes)

Slopes for each column: {'A': -0.0016136921226826213, 'AAL': -0.0027356854289976354, 'AAPL': -0.0008061483180459035, 'ABBV': -0.0008491277635299401, 'ABNB': -0.0024011644345169666, 'ABT': -0.0016300650033541045, 'ACGL': -0.00035702620150230907, 'ACN': -0.0005807759762845972, 'ADBE': -0.0006950423877214918, 'ADI': -0.0030765286915304343, 'ADM': 0.0005846581791496586, 'ADP': -0.0003603198182381635, 'ADSK': -0.0007778683027096087, 'AEE': -0.0009511190243153115, 'AEP': -0.0006946086640155241, 'AES': -0.0007818573994018079, 'AFL': 0.0006005241050868787, 'AIG': 0.0007713464818402254, 'AIZ': 0.0015408784636094094, 'AJG': 0.0005521802430962793, 'AKAM': 0.0009990130314015125, 'ALB': -0.003134930473873936, 'ALGN': -0.008108834007369706, 'ALK': -0.0028695341948164104, 'ALL': 0.0012829917937873848, 'ALLE': -0.0014105074853639685, 'AMAT': -0.0024350166616285552, 'AMCR': -0.001194920063401937, 'AMD': -0.0024093511171304157, 'AME': -0.001217750302396456, 'AMGN': 0.001356845242206392, 'AMP': -0.000240

In [12]:
slopes

{'A': -0.0016136921226826213,
 'AAL': -0.0027356854289976354,
 'AAPL': -0.0008061483180459035,
 'ABBV': -0.0008491277635299401,
 'ABNB': -0.0024011644345169666,
 'ABT': -0.0016300650033541045,
 'ACGL': -0.00035702620150230907,
 'ACN': -0.0005807759762845972,
 'ADBE': -0.0006950423877214918,
 'ADI': -0.0030765286915304343,
 'ADM': 0.0005846581791496586,
 'ADP': -0.0003603198182381635,
 'ADSK': -0.0007778683027096087,
 'AEE': -0.0009511190243153115,
 'AEP': -0.0006946086640155241,
 'AES': -0.0007818573994018079,
 'AFL': 0.0006005241050868787,
 'AIG': 0.0007713464818402254,
 'AIZ': 0.0015408784636094094,
 'AJG': 0.0005521802430962793,
 'AKAM': 0.0009990130314015125,
 'ALB': -0.003134930473873936,
 'ALGN': -0.008108834007369706,
 'ALK': -0.0028695341948164104,
 'ALL': 0.0012829917937873848,
 'ALLE': -0.0014105074853639685,
 'AMAT': -0.0024350166616285552,
 'AMCR': -0.001194920063401937,
 'AMD': -0.0024093511171304157,
 'AME': -0.001217750302396456,
 'AMGN': 0.001356845242206392,
 'AMP': -0

In [13]:
# Slope is our (y2-y1)[change of price] 
# divided by (x1-x2)[change of 1 year]
open_price = sp500.iloc[0] # first data entry
close_price = sp500.iloc[-1] # most recent data entry
price_difference = close_price - open_price
# Since our change of time is 1 year, im just going to divide by count of rows
slope = price_difference / len(sp500) * 100
slope

A        4.553334
AAL     -1.849935
AAPL    12.009972
ABBV     5.784782
ABNB          NaN
          ...    
YUM      3.247139
ZBH      0.823607
ZBRA     4.874801
ZION    -0.099172
ZTS      7.329728
Length: 501, dtype: float64

In [14]:
price_difference

A        57.280945
AAL     -23.272188
AAPL    151.085449
ABBV     72.772552
ABNB           NaN
           ...    
YUM      40.849014
ZBH      10.360977
ZBRA     61.324997
ZION     -1.247585
ZTS      92.207977
Length: 501, dtype: float64

In [15]:
def calculate_slope(data):
    x = np.arange(len(data))
    y = data.values
    slope, _ = np.polyfit(x, y, 1)
    return slope

In [16]:
calculate_slope(sp500)

array([ 6.13566679e-02, -1.35402781e-02,  1.21351692e-01,  8.69874757e-02,
                   nan,  3.13983505e-02,  3.56479022e-02,  1.42190330e-01,
        1.54278102e-01,  8.09208483e-02,  4.82991011e-02,  9.97034202e-02,
        4.02355575e-02,  1.89182008e-02,  1.25496059e-02,  7.36499349e-03,
        2.86754480e-02,  2.05343391e-02,  4.65957758e-02,  1.34600062e-01,
        1.22359531e-02,  1.64584377e-01,  5.71608507e-02, -1.32856085e-02,
        3.50811818e-02,  1.34883602e-02,  8.82763036e-02,  2.10846280e-03,
        6.92134080e-02,  6.49949973e-02,  7.08347595e-02,  2.08290558e-01,
        1.57339525e-02,  2.64430974e-02,  1.05642638e-01,  7.95754418e-02,
        1.51917135e-01,  2.48907662e-02,  1.83128531e-02,  8.84174805e-02,
        3.84115538e-02,  2.59336116e-02,  3.08737823e-03,  2.05645167e-02,
        1.90553895e-02,  4.89303144e-01,  8.39331904e-02,  3.82068381e-02,
        1.28793420e-01,  6.32854601e-02,  1.56927675e+00, -1.35447042e-01,
        7.24829580e-03, -

In [17]:

lag = 20

slope_52_week_trend = sp500.rolling(window=252).apply(calculate_slope, raw=False).shift(lag)
slope_52_week_trend

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py", line 6001, in __setattr__
    object.__getattribute__(self, name)
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py", line 669, in name
    return self._name
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py", line 5989, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'Series' object has no attribute '_name'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/6y/0l3pbhfs1bg6l9tz9cbfv4940000gp/T/ipykernel_13775/927857510.py", line 3, in <module>
    slope_52_week_trend = sp500.rolling(window=252).apply(calculate_slope, raw=Fal

1. **Slope of 52-week Trend Line (20-day lag):**
   - This indicator likely involves calculating the slope of the trend line based on the past 52 weeks of price data, with a 20-day lag. The slope provides information about the direction and strength of the trend.

In [18]:
lag = 20

momentum_factors = returns.rolling(window=lag).mean()
momentum_factors

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-29,0.010870,0.004820,0.005294,-0.000926,0.003706,0.004641,-0.002353,0.005818,0.007573,0.007612,...,-0.002004,0.000687,-0.001151,0.001817,0.005288,0.002172,0.004487,0.006458,0.007992,0.005844
2023-11-30,0.011199,0.005682,0.004510,0.000021,0.003152,0.004717,-0.003473,0.005191,0.005887,0.007189,...,-0.001077,0.000316,-0.000856,0.001781,0.005573,0.002002,0.005220,0.009265,0.007984,0.007865
2023-12-01,0.010804,0.006888,0.003815,0.000102,0.008249,0.004499,-0.001851,0.004685,0.004702,0.005738,...,-0.002261,0.000363,-0.002377,0.006769,0.006172,0.001486,0.003492,0.007885,0.008197,0.005434


In [19]:
what is lag? 

do we get all the factors?

do we the z scores, 

real value - expected value / std





SyntaxError: invalid syntax (137858137.py, line 1)

In [ ]:
# Step 3: Calculate the price momentum factors for each asset in your ETF

def calculate_momentum_factors(data, lag=20):
    # Factor 1: Slope of 52-week trend line (20-day lag)
    data['Slope_52Week'] = data['Close'].pct_change(252 - lag).rolling(window=20).mean() * 100

    # Factor 2: Percent above 260-day low (20-day lag)
    data['Percent_Above_260Day_Low'] = (data['Close'] - data['Low'].rolling(window=260 - lag).min()) / (data['High'].rolling(window=260 - lag).max() - data['Low'].rolling(window=260 - lag).min()) * 100

    # Factor 3: 4/52 Week Price Oscillator (20-day lag)
    data['Price_Oscillator'] = (data['Close'].rolling(window=4).mean() / data['Close'].rolling(window=52 - lag).mean() - 1) * 100

    # Factor 4: 39-week return (20-day lag)
    data['39Week_Return'] = data['Close'].pct_change(39 - lag) * 100

    # Factor 5: 51-week Volume Price Trend (20-day lag)
    data['Volume_Price_Trend'] = (data['Close'].pct_change() * data['Volume']).rolling(window=51 - lag).sum()

    return data[['Slope_52Week', 'Percent_Above_260Day_Low', 'Price_Oscillator', '39Week_Return', 'Volume_Price_Trend']]

# Apply the function to ETF data and asset data
#etf_momentum_factors = calculate_momentum_factors(data)
all_asset = {}

for asset in tickers_list:
    assets_momentum_factors = data[asset].apply(calculate_momentum_factors)

# Display the calculated price momentum factors
# print("ETF Momentum Factors:")
# print(etf_momentum_factors.head())
print("\nAsset Momentum Factors:")
print(assets_momentum_factors.head())

# # Displaying the signals for each asset
# for asset, signals in assets_momentum_factors.items():
#     print(f"Signals for {asset}:\n{signals}\n")



TypeError: 'float' object is not subscriptable

In [ ]:
# import yfinance as yf
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # Function to calculate the slope of a trend line
# def calculate_slope(data):
#     x = np.arange(len(data))
#     y = data.values
#     slope, _ = np.polyfit(x, y, 1)
#     return slope

# # Function to calculate the slope of the 52-week trend line with a 20-day lag
# def calculate_52_week_slope(ticker, start_date, end_date, lag):
#     # Download historical data
#     df = yf.download(ticker, start=start_date, end=end_date)
    
#     # Calculate the closing prices
#     closing_prices = df['Close']

#     # Calculate the slope with a 52-week window and a 20-day lag
#     slope_52_week_trend = closing_prices.rolling(window=52).apply(calculate_slope, raw=False).shift(lag)
    
#     return slope_52_week_trend

# # Example usage
# ticker = "AAPL"
# start_date = "2018-01-01"
# end_date = "2023-01-01"
# lag = 20

# # Calculate the 52-week slope with a 20-day lag
# slope_52_week_trend = calculate_52_week_slope(ticker, start_date, end_date, lag)

# # Plotting the results
# plt.figure(figsize=(10, 6))
# plt.plot(slope_52_week_trend, label='52-Week Slope (20-Day Lag)')
# plt.title(f'52-Week Slope of {ticker} Stock Price with a 20-Day Lag')
# plt.xlabel('Date')
# plt.ylabel('Slope')
# plt.legend()
# plt.show()
